# Week 10: Information Retrieval and Question Answering

This week we are looking at the tasks of information retrieval and question answering.  In the lab we will look at:
* finding relevant documents to a query expressed in terms of keywords
* extracting keywords from a query

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#preliminary imports
import pandas as pd
from itertools import zip_longest
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import math
import operator

import nltk
nltk.download('punkt')
nltk.download('stopwords')

We will be using the Stanford Question Answering Dataset (SQUAD) dataset in this lab, provided in the resources for this week.  For more information about the dataset and the ongoing competition, see the website:  https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/

The dataset is provided as 2 json files - one for development and one for training.  Lets read in the smaller development dataset using the json library.

In [ ]:
import json
filename="/content/drive/My Drive/NLE Notebooks/Week10Labs/Squad/dev-v2.0.json"

with open(filename,'r') as inputstream:
    devdata=json.load(inputstream)
    


The data is now stored in devdata as a deeply-nested dictionary structure.  The `data` field contains a list of dictionaries - one for each of the documents in the dataset.

In [ ]:
len(devdata['data'])

Each document dictionary has two key-value pairs.

In [ ]:
len(devdata['data'][0].keys())

In [ ]:
list(devdata['data'][0].keys())

The `paragraphs` field of a document dictionary is a further list of paragraphs!

In [ ]:
len(devdata['data'][0]['paragraphs'])

Each paragraph is a dictionary.  One field `context` contains the text of the paragraph as a String.  

In [ ]:
list(devdata['data'][0]['paragraphs'][0].keys())

In [ ]:
devdata['data'][0]['paragraphs'][0]['context']

The other field `qas` is a list of dictionaries.  Each dictionary contains a `question`, a list of possible `answers`, a unique `id` and a value for `is_impossible` which indicates whether the question is answerable or not given the `context`

In [ ]:
devdata['data'][0]['paragraphs'][0]['qas']

To make it easier to work with, lets turn this data structure into 2 simple lists containing the bits that we want.  The first will contain the `context` from the paragraphs and the other will contain the corresponding lists of questions and answers or `qas`.

In [ ]:
def get_text_qas(dataset):
    textlist=[]
    qalist=[]
    for document in dataset['data']:
        for para in document['paragraphs']:
            textlist.append(para['context'])
            qalist.append(para['qas'])
    
    
    return textlist,qalist
    
    
textlist,qalist=get_text_qas(devdata)

In [ ]:
print(textlist[1:2])
print(qalist[1:2])

## Keyword Search
In order to perform a keyword search of a document collection, the collection needs to be indexed by words.  The index is a mapping from keywords to lists of document ids containing that document.

Here, we will consider each paragraph as an individual document and its **id** is its position in the `textlist` list created above

### Exercise 1.1
Write a function `extract_keywords()` which takes a string and returns a set of keywords.  Make sure you carry out appropriate normalisation and filtering

### Exercise 1.2
Create a keyword index for the paragraphs in textlist.  

### Exercise 1.3
Write a class `docssearch` that is initialised with a list of strings, stores the list of strings and also creates an index.  Then write a method to search the keyword index for a single keyword and returns a list of paragraphs containing that keyword.  



The `merge` function below can be used to efficiently intersect two sorted lists.  It is based on the *merge* of *mergesort*.   You might want to test it out on some other lists.  Remember it will only work if the inputs are sorted.

In [ ]:
def merge(list1,list2):
    '''
    function to intersect 2 sorted lists
    '''
    pointer1=0
    pointer2=0
    merged=[]
    while pointer1<len(list1) and pointer2<len(list2):
        if list1[pointer1]==list2[pointer2]:
            merged.append(list1[pointer1])
            pointer1+=1
            pointer2+=1
            
        elif list1[pointer1]<list2[pointer2]:
            pointer1+=1
        else:
            pointer2+=1
            
    return merged

merge([1,5,7,8],[3,6,7,8,15])
        

### Exercise 1.4
Add a method `listsearch` that takes a list of keywords and returns a list of paragraphs which contains all of them.

Make sure your list of keywords is normalised and filtered in the same way as your paragraph text.

A search for paragraphs relevant to the keywords 'political' and 'kingdom' should return 2 paragraphs.  The first starts 'The Norman dynasty ...' and the second starts 'Throughout the 1980s and 1990s ...'

## Ranking Documents
A rank for a document can be calculated using the **sum** or the **product** of the tf-idf scores of each word in the query calculated over the documents

* Why weight the words by tf-idf?
* What difference does using the sum or the product make intuitively?

Alternatively, you can calculate the cosine similarity of the query to each document.

* What information does using cosine incorporate that is ignored by the simple sum or product measures?


### Exercise 2.1
Generate tf-idf representations of each of the paragraphs (look back to earlier labs to review how to do this).  Store these representations in the `docsearch` class as well.  Then use one of the ranking schemes described above to rank the documents returned by your `listsearch` method.

Using a product of tf-idf scores, you should find that the first ranked document for the search \['political','conquered'\] is para 1031 with a score of 114; whereas using a sum it is para 1031 with a score of 27.7

## Extensions

Implement one or more of the following extensions.

### Extension 1

In order to be able to answer natural language queries, it is also necessary to extract keywords from questions.

You can use the `input()` function to take in a query from keyboard.  Use this together with the functionality you have already developed to make a prototype of a system which returns the most relevant paragraph to a given query. 

### Extension 2

Find the sentences within a paragraph which are most relevant to a query.


### Extension 3

Build a classifier to determine whether a given query is answerable on the basis of a given relevant paragraph. 

You will need to use the `qalist` which has positive and negative examples of answerable questions for each paragraph.

An instance is then a paragraph and an answer pair which has been labelled answerable or unanswerable.  You can extract features from both the paragraph and the answer (e.g., bags-of-words) and then make a representation of the instance by applying some operation to the two vectors e.g., addition, subtraction, multiplication or concatenation.